In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df_train = pd.read_csv('train_pre_1.сsv')

In [12]:
df_train

,Unnamed: 0,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,...,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,...,1,1,33,7976,5,1143.192382,0,11,1,184966.930730
1,1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,...,1,1,46,10309,1,240.000000,1,16,1,300009.450063
2,2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,...,1,1,34,7759,0,229.000000,1,3,1,220925.908524
3,3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,...,1,1,23,5735,3,1084.000000,0,5,1,175616.227217
4,4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,...,1,1,35,5776,1,2078.000000,2,4,1,150226.531644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9982,9983,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,...,1,1,46,7960,6,350.000000,3,11,1,196684.316040
9983,9984,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,...,1,1,30,5562,0,1001.233832,0,5,0,189050.289571
9984,9985,5123,27,1.0,47.939008,34.857271,1.0,12,16.0,2015,...,1,1,2,629,1,964.926121,0,0,0,159143.805370
9985,9986,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,...,1,0,30,5048,9,325.000000,2,5,1,181595.339808


In [13]:
def weight_columns(df, y, put_model, rs=42):
    """
    df - датафрейм(первоначальный x)
    y - столбец который нужно предсказывать
    rs - random_state
    put_model - используемая модель

    Функция переберает столбцы таким образом чтобы вернуть те,
    при исопльзовании которых будет наивысший результат r2_score.
    Это будет означать что вот эти столбцы имеют действительное
    влияние на формирование предсказуемого столба, в рамках данной
    модели конечно.
    """
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import r2_score

    df_columns = list(df.columns)
    df_0 = list(df.columns[:2])

    x = df.loc[:, df_0]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=rs)
    model = put_model
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    r2 = r2_score(y_test, y_pred)

    power_columns = []+df_0
    for i in df.columns[2:]:
        c_power_columns = power_columns.copy()
        c_power_columns.append(i)
        x = df.loc[:, c_power_columns]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=rs)
        model = put_model
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        r2_new = r2_score(y_test, y_pred)

        if r2_new>r2:
            print(i, r2_new, "+")
            r2=r2_new
            power_columns.append(i)
        else:
            print(i, r2_new, "-")

    for i in df_0:
        c_power_columns = power_columns.copy()
        c_power_columns.remove(i)
        x = df.loc[:, c_power_columns]
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=rs)
        model = put_model
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        r2_new = r2_score(y_test, y_pred)

        print(i, r2_new)
        if r2_new>r2:
            print(i, r2_new, "+")
            r2=r2_new
            power_columns.remove(i)
        else:
            print(i, r2_new, "-")

    return [power_columns, r2]

In [14]:
from sklearn.linear_model import LinearRegression
x_0 = df_train.drop(columns=['Price'])
y = df_train['Price']
res = weight_columns(x_0, y, LinearRegression())

DistrictId 0.06695203046369735 +
Rooms 0.37444169770895763 +
Square 0.4224844195654298 +
LifeSquare 0.43166293104088804 +
KitchenSquare 0.4444295528902735 +
Floor 0.4604489578459777 +
HouseFloor 0.4617904111479595 +
HouseYear 0.46211587299627854 +
Ecology_1 0.46388184561636625 +
Ecology_2 0.4649505352249821 +
Ecology_3 0.4668001521108345 +
Social_1 0.4960577118892232 +
Social_2 0.4968018622349034 +
Social_3 0.49911823644078135 +
Healthcare_1 0.5031704792566547 +
Helthcare_2 0.5089088637886727 +
Shops_1 0.508561650826497 -
Shops_2 0.5139303837190952 +
Unnamed: 0 0.5138929929699037
Unnamed: 0 0.5138929929699037 -
Id 0.5136191510530521
Id 0.5136191510530521 -


In [15]:
res

[['Unnamed: 0',
  'Id',
  'DistrictId',
  'Rooms',
  'Square',
  'LifeSquare',
  'KitchenSquare',
  'Floor',
  'HouseFloor',
  'HouseYear',
  'Ecology_1',
  'Ecology_2',
  'Ecology_3',
  'Social_1',
  'Social_2',
  'Social_3',
  'Healthcare_1',
  'Helthcare_2',
  'Shops_2'],
 0.5139303837190952]

In [71]:
from sklearn.ensemble import RandomForestRegressor
x_0 = df_train.drop(columns=['Price'])
y = df_train['Price']
res = weight_columns(x_0, y, RandomForestRegressor(n_estimators=250, max_depth=18, random_state=42, max_features=7))

DistrictId 0.18620303116348802 +
Rooms 0.4457216462945629 +
Square 0.6323408087571591 +
LifeSquare 0.6215102709614322 -
KitchenSquare 0.637902384259601 +
Floor 0.6385752216442346 +
HouseFloor 0.646303807397618 +
HouseYear 0.672001961803518 +
Ecology_1 0.690706314396803 +
Ecology_2 0.6902224417987639 -
Ecology_3 0.6918618813795196 +
Social_1 0.7138835070483283 +
Social_2 0.7226399532418057 +
Social_3 0.7317300898086145 +
Healthcare_1 0.7308536040891818 -
Helthcare_2 0.7290992359959316 -
Shops_1 0.7300390574626425 -
Shops_2 0.7299110290847409 -
Unnamed: 0 0.728535737594618
Unnamed: 0 0.728535737594618 -
Id 0.7363092272894562
Id 0.7363092272894562 +


In [72]:
res

[['Unnamed: 0',
  'DistrictId',
  'Rooms',
  'Square',
  'KitchenSquare',
  'Floor',
  'HouseFloor',
  'HouseYear',
  'Ecology_1',
  'Ecology_3',
  'Social_1',
  'Social_2',
  'Social_3'],
 0.7363092272894562]

In [73]:
from sklearn.ensemble import GradientBoostingRegressor
x_0 = df_train.drop(columns=['Price'])
y = df_train['Price']
res = weight_columns(x_0, y, GradientBoostingRegressor(n_estimators=327, max_depth=7, random_state=42, max_features=14))

DistrictId 0.1628880770111103 +
Rooms 0.46015785739081716 +
Square 0.6403609251432146 +
LifeSquare 0.6431192744110393 +
KitchenSquare 0.6431351576794329 +
Floor 0.6645949924750543 +
HouseFloor 0.6729938370865935 +
HouseYear 0.692387970475805 +
Ecology_1 0.7130154971241955 +
Ecology_2 0.7193362669723442 +
Ecology_3 0.7180395965842336 -
Social_1 0.7356681566436529 +
Social_2 0.742763305584441 +
Social_3 0.7490182842288287 +
Healthcare_1 0.7349731128742774 -
Helthcare_2 0.7493422914286525 +
Shops_1 0.743005577805824 -
Shops_2 0.7436400184462442 -
Unnamed: 0 0.7455095799491391
Unnamed: 0 0.7455095799491391 -
Id 0.7500235520888577
Id 0.7500235520888577 +


In [74]:
res

[['Unnamed: 0',
  'DistrictId',
  'Rooms',
  'Square',
  'LifeSquare',
  'KitchenSquare',
  'Floor',
  'HouseFloor',
  'HouseYear',
  'Ecology_1',
  'Ecology_2',
  'Social_1',
  'Social_2',
  'Social_3',
  'Helthcare_2'],
 0.7500235520888577]

Выбор падает на GradientBoostingRegressor судя по самым высоким показателям

In [75]:
x_0 = df_train.drop(columns=['Price'])
x = df_train.loc[:, res[0]]
y = df_train['Price']
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=5000/(10000+5000), random_state=42)
model = GradientBoostingRegressor(n_estimators=327, max_depth=7, random_state=42, max_features=14)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2_score(y_test, y_pred)


0.7299608887747375

Расчитываю на такую точность прогноза при прогнозировании цен на test.csv
Однако учитывая что LifeSquare там cпрогнозирован менее точно показатели будут ниже

In [76]:
import json
with open('true_columns.json', 'w') as f:
    json.dump(res, f)